# Do experiment to make voice brighter

In [1]:
from pydub import AudioSegment
from pydub.effects import compress_dynamic_range, normalize
import pydub.scipy_effects as scipy_effects

In [2]:
shenhe_audio: AudioSegment = AudioSegment.from_file('./sound/adrenaline03.wav')
shenhe_audio

In [66]:
keqin_audio: AudioSegment = AudioSegment.from_file('./sound/adrenaline01.wav')
keqin_audio

## Solution 1 : Gain + Compress

In [4]:
def solution_1(input_audio: AudioSegment) -> AudioSegment:
    target_dbfs = 0
    gain = target_dbfs - input_audio.max_dBFS
    louder_audio = input_audio + gain

    compressed_audio: AudioSegment = compress_dynamic_range(louder_audio, 
        threshold=target_dbfs - 0.5,
        ratio=4.0, 
        attack=5.0, 
        release=50.0,
    )
    
    compressed_audio.export('output_audio.wav')
    
    return compressed_audio

solution_1(shenhe_audio)

## Solution 2: Gain + Normalize

In [68]:
def solution_2(input_audio: AudioSegment) -> AudioSegment:
    target_dbfs = 0
    gain = target_dbfs - input_audio.max_dBFS
    louder_audio = input_audio + gain
    normalized_audio: AudioSegment = normalize(louder_audio, headroom=0.05)
    return normalized_audio

solution_2(shenhe_audio)

## Solution 3: Normalize

In [69]:
def solution_3(input_audio: AudioSegment) -> AudioSegment:
    normalized_audio: AudioSegment = normalize(input_audio, headroom=0.05)
    return normalized_audio

solution_3(shenhe_audio)

In [70]:
solution_3(keqin_audio)

So I think, normalization is necessary. I need extra EQ to make voice brighter.

## Solution 4: EQ + Normalize

In [5]:
def solution_4(input_audio: AudioSegment, focus_freq=3000) -> AudioSegment:
    target_dbfs = 0
    gain = target_dbfs - input_audio.max_dBFS
    out = scipy_effects.eq(input_audio, focus_freq=focus_freq, gain_dB=gain, filter_mode="high_shelf", order=2)
    out: AudioSegment = normalize(out, headroom=0.05)
    out.export('output_audio.wav')
    return out
solution_4(shenhe_audio, focus_freq=200)

In [95]:
solution_4(keqin_audio, focus_freq=200)

In [98]:
shenhe_audio.set_frame_rate(shenhe_audio.frame_rate * 2)
shenhe_audio.export('test.wav')

<_io.BufferedRandom name='test.wav'>

## Explore method to make phone like effect

In [9]:
npc_audio: AudioSegment = AudioSegment.from_file('./sound/comeupstairs01.wav')
npc_audio

In [12]:
def phone_like(wav: AudioSegment, low_center=1600, high_center=300) -> AudioSegment:
    # audio = scipy_effects.low_pass_filter(wav, low_center)
    audio = scipy_effects.high_pass_filter(wav, high_center)
    audio = normalize(audio)
    return audio

phone_like(npc_audio)